In [1]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import torch.optim as optim
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from IPython.display import clear_output

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

2024-03-28 11:13:05.378069: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 11:13:05.378171: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 11:13:05.555903: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
class PairedImageDataset(Dataset):
    def __init__(self, rootA, rootB, transform=None):
        """
        Args:
            rootA (string): Directory with all the images in trainA.
            rootB (string): Directory with all the images in trainB.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.rootA = rootA
        self.rootB = rootB
        self.transform = transform

        # Assuming filenames in both folders are the same and in order
        self.filenames = sorted(os.listdir(rootA))

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        img_nameA = os.path.join(self.rootA, self.filenames[idx])
        imageA = Image.open(img_nameA).convert('RGB')
        
        img_nameB = os.path.join(self.rootB, self.filenames[idx])
        imageB = Image.open(img_nameB).convert('RGB')

        if self.transform:
            imageA = self.transform(imageA)
            imageB = self.transform(imageB)

        return imageA, imageB

In [3]:
class Encoder(nn.Module):
    def __init__(self, latent_dims, s_img, hdim, kernel_size = (4,4), stride =2, padding = 0):
        super(Encoder, self).__init__()

        self.layers = nn.ModuleList()

        # layer 1
        self.layers.append(
            nn.Conv2d(
                in_channels=hdim[0],
                out_channels=hdim[1],
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
            )
        )

        self.layers.append(nn.LeakyReLU(0.2))

        # the rest of the layers
        for in_out in range(1, (len(hdim) - 1)):
            self.layers.append(
                nn.Conv2d(
                    in_channels=hdim[in_out],
                    out_channels=hdim[(in_out + 1)],
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                )
            )

            self.layers.append(nn.BatchNorm2d(hdim[in_out + 1]))

            self.layers.append(nn.LeakyReLU(0.2))


    def forward(self, x):

        SkipConnections = []

        for layer in self.layers:
            # print(layer)
            x = layer.forward(x)
            SkipConnections.append(x)

        return x, SkipConnections

In [4]:
#decoder
class Decoder(nn.Module):
    def __init__(self, latent_dims, s_img, hdim_in, hdim_out, kernel_size = (4,4), stride =2, padding= 0):
        super(Decoder, self).__init__()

        self.layers = nn.ModuleList()

        # layers 1 to 3 with dropout
        for in_out in range(0, 3):
            self.layers.append(
                nn.ConvTranspose2d(
                    in_channels=hdim_in[in_out],
                    out_channels=hdim_out[(in_out + 1)],
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                )
            )

            self.layers.append(nn.BatchNorm2d(hdim_out[in_out + 1]))
            self.layers.append(nn.Dropout2d(0.5))
            self.layers.append(nn.ReLU())

        # the rest of the layers 
        for in_out in range(3, (len(hdim_in) - 1)):
            self.layers.append(
                nn.ConvTranspose2d(
                    in_channels=hdim_in[in_out],
                    out_channels=hdim_out[(in_out + 1)],
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                )
            )

            self.layers.append(nn.BatchNorm2d(hdim_out[in_out + 1]))
            self.layers.append(nn.Identity())
#             self.layers.append(nn.Dropout2d(0.5))
            self.layers.append(nn.ReLU())
        
        
            
    def forward(self, z, SkipConnections):
        
        EncoderIndex = 3/4
        SkipConnections.reverse()

        for i, layer in enumerate(self.layers):
            if i % 4 == 0 and i != 0:
                j = int(EncoderIndex * i)
                z = layer.forward(torch.add(z, SkipConnections[j]))
#                 z = layer.forward(torch.cat((z,SkipConnections[j]), 1)) 
                    
            else:
                z = layer.forward(z)
                
        z = torch.tanh(z)
#         z = z * 255
        return z

In [5]:
#Generator
class Generator(nn.Module):
    def __init__(self,
                latent_dims,
                s_img,
                hdim_e,
                hdim_d_input,
                hdim_d_output,
                kernel_size, 
                padding):
        super(Generator, self).__init__()

        self.encoder = Encoder(
            latent_dims=latent_dims,
            s_img=s_img,hdim=hdim_e,
            kernel_size= kernel_size,
            padding= padding)
        self.decoder = Decoder(
            latent_dims=latent_dims,
            s_img=s_img, 
            hdim_in=hdim_d_input, 
            hdim_out=hdim_d_output,
            kernel_size=kernel_size, 
            padding=padding)

    def forward(self, x):

        z, skipConnections = self.encoder(x)
        # print(f"the shape of encoder is {z.shape}")
        y = self.decoder(z, skipConnections)

        return y

In [6]:
class Discriminator(nn.Module):
    
    def __init__(self, latent_dims, s_img, hdim, kernel_size=(4, 4), stride=2):
        
        super(Discriminator, self).__init__()

        ########################################################################
        #    Create the necessary layers                                 #
        ########################################################################

        self.layers = nn.ModuleList()
        
        # Input layer dim -- down1
        self.layers.append(nn.Conv2d(in_channels=6, out_channels=64, kernel_size=kernel_size, stride=2, padding=1))

        # Hidden to hidden convolution -- down2 and down 3
        for i in range(0, 2):
            self.layers.append(nn.Conv2d(in_channels=hdim[i],
                                             out_channels=hdim[i + 1],
                                             kernel_size=kernel_size, stride = stride, padding=1))

        # Pad with zeroes
        self.layers.append(nn.ZeroPad2d(padding=(1,1,1,1)))

        # Conv2D
        self.layers.append(nn.Conv2d(in_channels=hdim[3],
                                             out_channels=hdim[4],
                                             kernel_size=kernel_size, stride = 1))

        # Batchnorm
        self.layers.append(nn.BatchNorm2d(hdim[4]))

        # Zeropad2
        self.layers.append(nn.ZeroPad2d(padding=(1,1,1,1)))

        #Conv2D 2
        self.layers.append(nn.Conv2d(in_channels=hdim[5],
                                             out_channels=hdim[6],
                                             kernel_size=kernel_size, stride = 1))

        self.Leakyrelu = nn.LeakyReLU(0.2)
        
    def forward(self, x):

        for n_layer, layer in enumerate(self.layers):
            ## The fourth layer first has a batchnorm and then a Leakyrelu
            if n_layer != 4:
                x = self.Leakyrelu(layer(x))
            else:
                x = layer(x)
        return x
        

In [7]:
def train_generator(train_loader, net, optimizer, criterion, pin_memory , device = 'cpu'):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
        device: whether the network runs on cpu or gpu
    """

    avg_loss = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        
        # convert the inputs to run on GPU if set
        if device != 'cpu':
            inputs, labels = inputs.cuda(non_blocking=pin_memory), labels.cuda(non_blocking=pin_memory)
            

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels) * 100 #+ F.binary_cross_entropy(outputs, torch.ones(outputs.shape, device=device))
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += loss

    return avg_loss/len(train_loader), outputs

def try_gpu():
    """
    If GPU is available, return torch.device as cuda:0; else return torch.device
    as cpu.
    """
    if torch.cuda.is_available():
        device = torch.device('cuda')
    else:
        device = torch.device('cpu')
    return device

In [29]:
def train_discriminator(train_loader, net, optimizer, criterion, pin_memory , device = 'cpu'):
    """
    Trains network for one epoch in batches.

    Args:
        train_loader: Data loader for training set.
        net: Neural network model.
        optimizer: Optimizer (e.g. SGD).
        criterion: Loss function (e.g. cross-entropy loss).
        device: whether the network runs on cpu or gpu
    """
    
    # get the inputs; data is a list of [inputs, labels]
    labels = torch.empty(output.shape)
    x = torch.empty((1, 6, 256, 256))
    
    avg_loss = 0

    # iterate through batches
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        print(data.shape)
        img_gen, input_img = data
        
        x[i, :, :, :] = torch.cat((img_gen, input_img), dim=0)
        
        labels[i, 0] = torch.zeros(30, 30)
        
        
        # convert the inputs to run on GPU if set
        if device != 'cpu':
            inputs, labels = inputs.cuda(non_blocking=pin_memory), labels.cuda(non_blocking=pin_memory)
            

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)

        loss = criterion(outputs, labels)  #+ F.binary_cross_entropy(outputs, torch.ones(outputs.shape, device=device))
        loss.backward()
        optimizer.step()

        # keep track of loss and accuracy
        avg_loss += loss

    return avg_loss/len(train_loader)



In [30]:
if __name__ == "__main__":
    # show_images()
    torch.cuda.empty_cache()
    num_workers = 4
    pin_memory = True
    batch_size = 32
    n_samples, in_channels, s_img, latent_dims, padding = 1, 3, 256, 512,1
    hdim_e = [3, 64, 128, 256, 512, 512, 512, 512, 512] #choose hidden dimension encoder
    hdim_d_output = [512, 512, 512, 512, 512, 256, 128, 64, 3]
#     hdim_d_input = [512, 1024, 1024, 1024, 1024, 512, 256, 128, 3] #choose hidden dimension decoder
    hdim_d_input = hdim_d_output
    in_channels_gen = 6 # 6 for two images
    hdim_dis = [64, 128, 256, 256, 512, 512, 1] #choose hidden dimension discriminator
    output_shape = (n_samples, 1, 30, 30)
    
    kernel_size = (4,4)

   # Define the transformation pipeline with added jitter
    transform = transforms.Compose([
        transforms.Resize(256, interpolation=transforms.InterpolationMode.NEAREST),  # Resize to 256x256 using nearest neighbor method
        transforms.RandomCrop(256),  # Apply random cropping to introduce jitter
        transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
        # Random jitter through changes in brightness, contrast, saturation, and hue
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.02),
        # Random affine transformations for scaling and rotation
        transforms.RandomAffine(degrees=10, scale=(0.95, 1.05)),
        transforms.ToTensor(),  # Convert images to tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize to [-1, 1]
    ])

    # Initialize the dataset
    paired_dataset = PairedImageDataset(rootA=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainA',
                                        rootB=r'/kaggle/input/euvp-dataset/EUVP/Paired/underwater_dark/trainB',
                                        transform=transform)

    # Initialize DataLoader
    EUVP_data = DataLoader(
        paired_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory)

   # Fetch a single batch
    images_a, images_b = next(iter(EUVP_data))
#     _ , img_x = EUVP_data

    #Set the number of dimensions of the latent space
    # latent_dims = [2,3]
    s_img = np.size(images_a[1][0], axis = 1) #get image size (height = width) from a data sample

    #initialize generator model
    model_gen = Generator(latent_dims=latent_dims,
                        s_img=s_img,
                        hdim_e=hdim_e, 
                        hdim_d_input=hdim_d_input,
                        hdim_d_output=hdim_d_output, 
                        padding=padding,
                        kernel_size=kernel_size)
    #intit dis
    model_dis = Discriminator(latent_dims, s_img, hdim_dis)

    # Create a writer to write to Tensorboard
    writer = SummaryWriter()

    #Create instance of Autoencoder
    device = try_gpu()
    print(device)
    
    if torch.cuda.is_available():
        model_gen= nn.DataParallel(model_gen)
        model_gen.to(device)
        model_dis = nn.DataParallel(model_dis)
        model_dis.to(device)
    # AE2 = Generator(latent_dims[0], s_img, hdim = hdim).to(device) #2-dimensional latent space
    # AE3 = Generator(latent_dims[1], s_img, hdim = hdim).to(device) #3-dimensional latent space

    # Create loss function and optimizer
    criterion_gen = F.l1_loss 
    criterion_dis = nn.BCELoss()
    
    optimizer_gen = optim.Adam(model_gen.parameters(), lr=2e-4, betas=(0.5, 0.999))
    optimizer_dis = optim.Adam(model_dis.parameters(), lr=2e-4, betas=(0.5, 0.999))

    # Set the number of epochs to for training
    epochs = 20
    
    plt.ion()  # Enable interactive mode
    
    losses_gen = []
    losses_dis = [] 
    
    data_set_dis = []
    for i in range(len(outputs)):
        data_set_dis.append((outputs[i], images_b[i].cuda(non_blocking=pin_memory)))
        
    
    for epoch in tqdm(range(epochs)):  # loop over the dataset multiple times
        # Train on data
        train_loss_gen, outputs = train_generator(EUVP_data, model_gen, optimizer_gen, criterion_gen, pin_memory, device)
#         print(f'outputs ={outputs.shape}')
#         print(f'images_b ={img_x.shape}')
        train_loss_dis = train_discriminator(data_set_dis, model_dis, optimizer_dis, criterion_gen, pin_memory, device)
        # Write metrics to Tensorboard
        writer.add_scalars("Loss", {'Train': train_loss_gen}, epoch)

        losses_gen.append(train_loss_gen.item())
        losses_dis.append(train_loss_dis.item())
        
        # Visualization code
        clear_output(wait=True)
        plt.figure(figsize=(10, 5))
        plt.plot(losses_gen, label='Training Loss')
        plt.plot(losses_dis, label='Training Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss Over Time')
        plt.legend()
        plt.show()

    plt.ioff()  # Disable interactive mode
    writer.close()
    



cuda


  0%|          | 0/20 [00:48<?, ?it/s]


NameError: name 'output' is not defined

In [ ]:

# make a folder
count = 0
image_counter = 10
path = '/kaggle/working/test'
path_loop = None
# Initialize DataLoader
EUVP_data = DataLoader(paired_dataset, batch_size=5, shuffle=False, num_workers=4)

print("start printing")
# Store somep images
for i, data in enumerate(EUVP_data):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    y_out = model.forward(inputs)
#     y_out = torch.tanh(y_out)

    for x in range(len(inputs) - 1):
        path_loop = f"{path}/test{count}"
        if not (os.path.exists(path_loop) and os.path.isdir(path_loop)):
            os.makedirs(path_loop)

        name = (f'{path_loop}/image_in{count}.jpg')
        save_image(inputs[x], name)
        name = (f'{path_loop}/image_out{count}.jpg')
        save_image(y_out[x], name)
        name = (f'{path_loop}/image_truth{count}.jpg')
        save_image(labels[x], name)
        count += 1

        if (count == image_counter):
            break
    break


In [ ]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/working/runs

In [ ]:
 %reload_ext tensorboard

In [ ]:
!kill 292